In [36]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

from keras.models import Sequential
from keras.layers import Bidirectional, Embedding, LSTM, Dense, RepeatVector

In [37]:
with open("Tamil-English-Dataset/Dataset/data.en1") as f:
    data_en = f.readlines()
    data_en = data_en[:1000]

with open("Tamil-English-Dataset/Dataset/data.ta1") as f:
    data_t = f.readlines()
    data_t = data_t[:1000]


In [ ]:
data_t

In [39]:
with open('data_t.txt', 'w', encoding='utf-8') as f:
    f.writelines(data_t)

In [40]:
with open('data_en.txt', 'w', encoding='utf-8') as f:
    f.writelines(data_en)

In [41]:
import fasttext
model_t = fasttext.train_unsupervised('data_t.txt', model='skipgram')

In [42]:
word_vector_dict = {}
for word, vector in zip(word_index, word_vectors):
    word_vector_dict[word] = vector

In [43]:
word_vectors = model_t.get_input_matrix()
print(len(word_vectors))

2000375


In [44]:
word_index = model_t.get_words()
print(len(word_index))

375


In [47]:
model_en = fasttext.train_unsupervised('data_en.txt', model='skipgram')

In [48]:
word_vectors = model_en.get_input_matrix()
print(len(word_vectors))

2000576


In [49]:
word_index = model_en.get_words()
print(len(word_index))

576


In [50]:
def pad_sequences(data, maxlen):
    data = list(map(lambda x: x + ['<PAD>'] * (maxlen - len(x)), data))
    return data

data_t_split = list(map(lambda x: x.split(), data_t))
data_t_pad = pad_sequences(data_t_split, maxlen=100)

data_en_split = list(map(lambda x: x.split(), data_en))
data_en_pad = pad_sequences(data_en_split, maxlen=100)

In [51]:
def get_embed(model,sentence):
    embed = []
    for word in sentence:
        embed.append(model.get_word_vector(word))

    embed = np.stack(embed, axis=0).astype(np.float32)
    return embed

In [54]:
t_embed = []
for sentence in data_t_pad:
    t_embed += [get_embed(model_t,sentence)]

en_embed = []
for sentence in data_en_pad:
    en_embed += [get_embed(model_en,sentence)]


In [55]:
t_embed = np.stack(t_embed, axis=0).astype(np.float32)
en_embed = np.stack(en_embed, axis=0).astype(np.float32)


In [58]:
from keras.callbacks import EarlyStopping

In [142]:
model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dense(100, activation='relu'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.build(input_shape=(None, 100, 100))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 100, 256)         234496    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 100, 256)         394240    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 100, 256)         394240    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 100, 256)         394240    
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 100, 256)        

In [143]:
model.fit(t_embed, en_embed, epochs=10, callbacks=EarlyStopping(monitor='accuracy', min_delta=1e-4, patience=2))

Epoch 1/10
32/32 [==============================] - 22s 266ms/step - loss: 0.0032 - accuracy: 0.1573
Epoch 2/10
32/32 [==============================] - 5s 145ms/step - loss: 0.0029 - accuracy: 0.4148
Epoch 3/10
32/32 [==============================] - 4s 139ms/step - loss: 0.0029 - accuracy: 0.2205
Epoch 4/10
32/32 [==============================] - 4s 136ms/step - loss: 0.0029 - accuracy: 0.2229


In [ ]:
model.fit(t_embed, en_embed, epochs=10, callbacks=EarlyStopping(monitor='accuracy', min_delta=1e-4, patience=2))

Epoch 1/10
32/32 [==============================] - 10s 89ms/step - loss: 0.0037 - accuracy: 0.1236
Epoch 2/10
32/32 [==============================] - 3s 80ms/step - loss: 0.0030 - accuracy: 0.1753
Epoch 3/10
32/32 [==============================] - 2s 76ms/step - loss: 0.0029 - accuracy: 0.2162
Epoch 4/10
32/32 [==============================] - 3s 79ms/step - loss: 0.0029 - accuracy: 0.2232
Epoch 5/10
32/32 [==============================] - 2s 69ms/step - loss: 0.0029 - accuracy: 0.2206
Epoch 6/10
32/32 [==============================] - 2s 71ms/step - loss: 0.0028 - accuracy: 0.2239
Epoch 7/10
32/32 [==============================] - 2s 73ms/step - loss: 0.0028 - accuracy: 0.2223
Epoch 8/10
32/32 [==============================] - 2s 69ms/step - loss: 0.0028 - accuracy: 0.2211


In [85]:
def get_word(vector, model):
    vectors = [model.get_word_vector(word) for word in model.get_words()]
    vectors = np.stack(vectors, axis=0).astype(np.float32)
    similarity = vectors @ vector / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(vector))
    most_similar_index = np.argmax(similarity)
    words = model_en.get_words()
    most_similar_word = words[most_similar_index]

    return most_similar_word

In [127]:
test_input = data_t[69]
test_input = test_input.split()

In [128]:
test_input_embed = get_embed(model_t, test_input)

In [129]:
test_pred = model.predict(np.expand_dims(test_input_embed, axis=0))

1/1 [==============================] - 1s 1s/step


In [144]:
y_pred = []
for word in test_pred[0]:
    y_pred.append(get_word(word, model_en))

y_pred = ' '.join(y_pred)

In [131]:
y_pred

'if if if if if if if if if if if if if if if if if if if if if if if if if if if'

In [146]:
y_pred

'if if if if if if if if if if if if if if if if if if if if if if if if if if if'